In [1]:
# Load packages
import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

from langchain_openai import OpenAI

# CONSTANTS
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define Model
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo-instruct",
    temperature=0.7,
    max_tokens=256
)

## LLM Chain

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

article = """
Chef and Owner Angela Kora was a mechanical engineer before bringing her attention to detail to the baking world. 
The menu at Ethos Bakery & Café is designed around fresh, local fare. 
The bakery even mills their own grain on site. 
Guy was impressed by the over-the-top Ethos Burger which is served on a house-made croissant bun with a bacon and braised leek topping. 
That one of the most outside-of-the-box burgers I have ever had on DDD,” said Guy.
"""

fact_extraction_template = """Extract key facts from the article. Don't include opinions. Give each fact a number and keep it short.\n 
{article_input}
"""

fact_extraction_chain_prompt = PromptTemplate(
    input_variables=["article_input"],
    template=fact_extraction_template,
)

fact_extraction_chain = LLMChain(
    llm=llm,
    prompt=fact_extraction_chain_prompt,
)

facts = fact_extraction_chain.run(article)

print(facts)


1. Angela Kora was previously a mechanical engineer before becoming a chef and owner of Ethos Bakery & Café.
2. The menu at Ethos Bakery & Café features fresh and local food options.
3. The bakery mills their own grain on site.
4. The Ethos Burger is served on a house-made croissant bun and topped with bacon and braised leeks.
5. Guy Fieri was impressed by the unique and creative Ethos Burger on his show "Diners, Drive-Ins, and Dives."


In [10]:
headline_writing_template = """
Given the following facts, write a catchy headline for the same:\n
{extracted_facts}
"""


headline_writing_chain_prompt = PromptTemplate(
    input_variables=["extracted_facts"],
    template=headline_writing_template
)

headline_writing_chain = LLMChain(
    llm=llm,
    prompt=headline_writing_chain_prompt
)

headline = headline_writing_chain.run(facts)
print(headline)


"From Engineer to Chef: Meet the Mastermind Behind the Mouth-Watering Ethos Burger, Praised by Guy Fieri on 'Diners, Drive-Ins, and Dives'!"


## LLM Sequential Chains

In [12]:
from langchain.chains import SequentialChain, SimpleSequentialChain

headline_extraction_chain = SimpleSequentialChain(
    chains=[fact_extraction_chain, headline_writing_chain]
)

extracted_headline = headline_writing_chain.run(article)
print(extracted_headline)

"From Mechanical Engineer to Master Chef: Meet the Creator of the Out-of-the-Box Ethos Burger at Ethos Bakery & Café!" 



## LLM Math Chain

In [22]:
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain(llm=llm, verbose=True)
answer = llm_math_chain.run("What is 14560 added to 500?")
print(answer)

answer = llm_math_chain.run("John had 50 apples. He bought 3 Mangoes. How many apples does he have?")

print(answer)



> Entering new LLMMathChain chain...
What is 14560 added to 500?```text
14560 + 500
```
...numexpr.evaluate("14560 + 500")...

Answer: 15060
> Finished chain.
Answer: 15060


> Entering new LLMMathChain chain...
John had 50 apples. He bought 3 Mangoes. How many apples does he have?```text
50 + 3
```
...numexpr.evaluate("50 + 3")...

Answer: 53
> Finished chain.
Answer: 53


## CoT

In [24]:
cot_template = """
You are an expert mathematical reasoner. Given a mathematical question, reason and return a numbered list of 
arithmetic steps to be evaluted: \n
{input_question}
"""

cot_prompt = PromptTemplate(
    input_variables=["input_question"],
    template=cot_template
)

cot_chain = LLMChain(
    llm=llm,
    prompt=cot_prompt
)


complex_math_chain = SimpleSequentialChain(
    chains=[cot_chain, llm_math_chain]
)

question = "John had 50 apples. He bought 3 Mangoes. How many apples does he have?"
final_answer = complex_math_chain.run(question)
print(answer)



> Entering new LLMMathChain chain...

1. Start with the number of apples John originally had: 50 apples.
2. Add the number of mangoes he bought: 50 apples + 3 mangoes.
3. Convert the 3 mangoes into apples: 3 mangoes = 3 x 2 apples = 6 apples.
4. Add the number of apples from the mangoes to the original number of apples: 50 apples + 6 apples = 56 apples.
5. Therefore, John now has 56 apples.
```text
50 + (3 * 2)
```
...numexpr.evaluate("50 + (3 * 2)")...

Answer: 56
> Finished chain.
Answer: 53


## PAL Chain

In [29]:
from langchain_experimental.pal_chain import PALChain


question = "John had 50 apples. He bought 3 Mangoes. How many apples does he have?"
pal_chain = PALChain.from_math_prompt(llm=llm, verbose=True)
answer = pal_chain.run(question)
print(answer)



> Entering new PALChain chain...
def solution():
    """John had 50 apples. He bought 3 Mangoes. How many apples does he have? Think step by step."""
    apples_initial = 50
    mangoes_bought = 3
    apples_after = apples_initial - mangoes_bought
    result = apples_after
    return result

> Finished chain.
47
